In [1]:
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")
model.cuda()
model.eval();

Use HM Bias:  -2.19


In [2]:
import pickle
pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    dummy_input = pickle.load(handle)
        

In [3]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = model.neck.find_centers(dummy_input)

In [4]:
import tensorrt as trt
print(trt.__version__)

8.5.1.7


In [5]:
from tools.trt_utils import load_engine
centerFinder_engine_path = '/data/centerformer/work_dirs/partition/engine/findCenter_sanitized_fp32.trt'

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
cf_engine = load_engine(centerFinder_engine_path, TRT_LOGGER)
cf_context = cf_engine.create_execution_context()

In [6]:
to_device = torch.device("cuda")
ct_feat_tensor = torch.zeros((1, 3000, 256), dtype=torch.float32, device=to_device)
center_pos_embedding_tensor = torch.zeros((1, 3000, 256), dtype=torch.float32, device=to_device)
out_scores_tensor = torch.zeros((6, 1, 500), dtype=torch.float32, device=to_device)
out_labels_tensor = torch.zeros((6, 1, 500), dtype=torch.int32, device=to_device)
out_orders_tensor = torch.zeros((6, 1, 500), dtype=torch.int32, device=to_device)
out_masks_tensor = torch.zeros((6, 1, 500), dtype=torch.bool, device=to_device)
IO_tensors = {
    "inputs" :
    {'input_tensor': dummy_input},
    "outputs" :{
        'out_labels': out_labels_tensor,
        'ct_feat': ct_feat_tensor, 
        'center_pos_embedding': center_pos_embedding_tensor,
        'out_scores': out_scores_tensor, 
        'out_orders': out_orders_tensor, 
        'out_masks': out_masks_tensor}
}

In [7]:
from tools.trt_utils import run_trt_engine

run_trt_engine(cf_context, cf_engine, IO_tensors)

In [8]:
print(out_scores[0][0][:10])
print(out_scores_tensor[0][0][:10])#tensorRT

tensor([0.8610, 0.8600, 0.8393, 0.8083, 0.8044, 0.7641, 0.7413, 0.7288, 0.7048,
        0.6847], device='cuda:0')
tensor([0.8610, 0.8600, 0.8393, 0.8083, 0.8044, 0.7641, 0.7413, 0.7288, 0.7048,
        0.6847], device='cuda:0')


In [9]:
print(ct_feat[0][0][:20])
print(ct_feat_tensor[0][0][:20])

tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7531, 0.0000, 0.0000,
        0.0000, 0.8260], device='cuda:0')
tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7531, 0.0000, 0.0000,
        0.0000, 0.8260], device='cuda:0')


In [10]:
print(out_orders[0][0][:20])
print(out_orders_tensor[0][0][:20])

tensor([49866, 62040, 59169, 57018, 61680, 98445, 59170, 50226, 30777, 57017,
        59529, 98446, 59530, 27541, 62041, 49867, 61679, 30776, 31137, 49506],
       device='cuda:0', dtype=torch.int32)
tensor([49866, 62040, 59169, 57018, 61680, 98445, 59170, 50226, 30777, 57017,
        59529, 98446, 59530, 27541, 62041, 49867, 61679, 30776, 31137, 49506],
       device='cuda:0', dtype=torch.int32)


In [11]:
torch.testing.assert_close(ct_feat_tensor, ct_feat, rtol=1e-03, atol=1e-05)

AssertionError: Tensor-likes are not close!

Mismatched elements: 1784 / 768000 (0.2%)
Greatest absolute difference: 1.414588212966919 at index (0, 1860, 115) (up to 1e-05 allowed)
Greatest relative difference: inf at index (0, 401, 8) (up to 0.001 allowed)